# Resize image for training

## -> This code does not run and is only meant for comparing external datasets, feel free to modify it for it to work. <-

In [1]:
from PIL import Image, UnidentifiedImageError
import os
import pandas as pd
import sys
import concurrent
import json

sys.path.insert(0, "../../")
from countryconverter import convert_coordinates_to_country_names
from region_enricher import RegionEnricher

region_enricher = RegionEnricher()
gdf = region_enricher.load_enriched_geojson()

In [2]:
def add_country(df):
    lat_lon_tuples = [(lat, lon) for lat, lon in zip(df["lat"], df["lon"])]
    df["country_name"], df["country_code"] = convert_coordinates_to_country_names(lat_lon_tuples)
    return df


def add_region(df):
    lat_lon_tuples = [(lat, lon) for lat, lon in zip(df["lat"], df["lon"])]
    df["regions"], df["is_in_region"] = region_enricher.get_region_from_points(gdf, lat_lon_tuples)
    return df


# convert the data to single json file per file, with the following columns: the name of the file is the {ID}.json and convert it to utf-8
# example: {"coordinates": [45.96841812133789, -66.80126190185547], "country_name": "Canada", "country_code": "CA", "regions": [["Canada_New Brunswick_CAN-684", "POINT (-66.37845 46.625711)"], ["United States_Maine_USA-3561", "POINT (-69.232905 45.388285)"], ["Canada_Prince Edward Island_CAN-687", "POINT (-63.249697 46.400597)"], ["Canada_Nova Scotia_CAN-685", "POINT (-63.310821 45.153825)"], ["United States_New Hampshire_USA-3538", "POINT (-71.57834 43.68971)"]], "is_in_region": "False"}
def convert_to_json(df, name):
    for i, row in df.iterrows():
        data = {}
        data["coordinates"] = [row["lat"], row["lon"]]
        data["country_name"] = row["country_name"]
        data["country_code"] = row["country_code"]
        data["regions"] = row["regions"]
        data["is_in_region"] = row["is_in_region"]
        with open(f'E:\\dspro2\\Paper_data\\{name}\\jsons\\{row["ID"]}.json', "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False)

In [3]:
input_file = "E:\\dspro2\\Paper_data\\yfcc4k\\info.txt"
output_file = "E:\\dspro2\\Paper_data\\yfcc4k\\info.csv"

In [4]:
# convert txt to csv line by line splitting by space and saving as csv with comma separator columnnames (ID, lat, lon)
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()
    with open(output_file, "w") as f:
        f.write("ID,lat,lon\n")
        for line in lines:
            f.write(line.replace(" ", ","))
            f.write("\n")

# load csv
df = pd.read_csv(output_file, sep=",")
print(df.head())

            ID      lat       lon
0  10201275523  43.6497  -79.3656
1   7289030198  40.7401  -73.9855
2   4572998878  42.3372  -71.0478
3   2764520341  45.4394   12.3334
4   4224177264  22.2808  114.1720


In [5]:
df = add_country(df)
df = add_region(df)
print(df.head())

df.to_csv(output_file, index="ID", encoding="utf-8")

convert_to_json(df, "yfcc4k")

Loading formatted geocoded file...
Finished checking if points are in regions.  Now finding nearest regions...
Finished finding nearest regions.
            ID      lat       lon   country_name country_code  \
0  10201275523  43.6497  -79.3656         Canada           CA   
1   7289030198  40.7401  -73.9855  United States           US   
2   4572998878  42.3372  -71.0478  United States           US   
3   2764520341  45.4394   12.3334          Italy           IT   
4   4224177264  22.2808  114.1720      Hong Kong           HK   

                                             regions  is_in_region  
0  [[United States_Pennsylvania_USA-3560, POINT (...          True  
1  [[United States_New Jersey_USA-3558, POINT (-7...          True  
2  [[United States_Massachusetts_USA-3513, POINT ...          True  
3  [[Italy_Venezia_ITA-5461, POINT (12.480684 45....         False  
4  [[Hong Kong_Wan Chai_HKG-5154, POINT (114.1772...         False  


In [6]:
input_img2gps = "E:\\dspro2\\Paper_data\\im2gps3ktest\\im2gps3k_places365.csv"
output_img2gps = "E:\\dspro2\\Paper_data\\im2gps3ktest\\im2gps3k_places_changed.csv"

In [7]:
df = pd.read_csv(input_img2gps, sep=",", encoding="utf-8")
# only get the columns (IMG_ID, LAT, LON)
df = df[["IMG_ID", "LAT", "LON"]]
# rename columns to (ID, lat, lon)
df.columns = ["ID", "lat", "lon"]

# remove .jpg from ID
df["ID"] = df["ID"].str.replace(".jpg", "")

# add country and region
df = add_country(df)
df = add_region(df)

print(df.head())

# save as csv
df.to_csv(output_img2gps, sep=",", index="ID", encoding="utf-8")

convert_to_json(df, "im2gps3ktest")

Finished checking if points are in regions.  Now finding nearest regions...
Finished finding nearest regions.
                                        ID        lat         lon  \
0  1000269685_e60e9cdfb4_1125_78841376@N00  32.325436  -64.764404   
1  1000304467_1a75a200b1_1296_78841376@N00  32.325436  -64.764404   
2  1001048550_8e4b47d165_1051_78841376@N00  32.325436  -64.764404   
3  1005977048_5ccf8b05d3_1201_91728102@N00  29.976052  122.390356   
4  1008804117_ce4e6fef8a_1349_97522422@N00  46.478536   30.758714   

  country_name country_code  \
0      Bermuda           BM   
1      Bermuda           BM   
2      Bermuda           BM   
3        China           CN   
4      Ukraine           UA   

                                             regions  is_in_region  
0  [[Bermuda_Devonshire_BMU-5136, POINT (-64.7593...         False  
1  [[Bermuda_Devonshire_BMU-5136, POINT (-64.7593...         False  
2  [[Bermuda_Devonshire_BMU-5136, POINT (-64.7593...         False  
3  [[China_S

Use pillow to resize images to 128x128 for training.
(images larger than 128x128 will be resized to 128x128, images smaller than will be ignored)

In [8]:
# risize an image using pillow
def resize_image(image, size, image_path):
    try:
        return image.resize(size)
    except OSError as e:
        print(e)
        if "image file is truncated" in e.__repr__() or (e.strerror is not None and "image file is truncated" in e.strerror):
            print("Image file is truncated. Skipping file: " + os.path.basename(image_path))
            return None
        raise e
    except SyntaxError as e:
        print(e)
        if "broken PNG file" in e.__repr__() or (e.strerror is not None and "broken PNG file" in e.strerror):
            print("Broken PNG file. Skipping file: " + os.path.basename(image_path))
            return None
        raise e


# save an image using pillow
def save_image(image, image_path):
    image.save(image_path)


# convert an image to grayscale using pillow
def convert_to_grayscale(image):
    return image.convert("L")

In [9]:
allow_env = True
path_to_images = "E:\\dspro2\\Paper_data\\yfcc4k\\yfcc4k"
path_to_processed_images = "E:\\dspro2\\Paper_data\\yfcc4k\\image_processed"
num_workers = 16

In [10]:
image_paths = [os.path.join(path_to_images, f) for f in os.listdir(path_to_images) if os.path.isfile(os.path.join(path_to_images, f))]

print(f"{len(image_paths)} files")

4536 files


In [11]:
# first crop the images to the 16:9 aspect ratio
def crop_image(image, image_path):
    width, height = image.size
    if width / height > 16 / 9:
        new_width = int(height * 16 / 9)
        left = (width - new_width) / 2
        right = (width + new_width) / 2
        top = 0
        bottom = height
    else:
        new_height = int(width * 9 / 16)
        left = 0
        right = width
        top = (height - new_height) / 2
        bottom = (height + new_height) / 2
    return image.crop((left, top, right, bottom))

In [12]:
# process the images
def process_image(image_path, path_to_processed_image):
    try:
        image = Image.open(image_path)
        image = crop_image(image, image_path)
        image = resize_image(image, (130, 80), image_path)
        save_image(image, os.path.join(path_to_processed_image, os.path.basename(image_path)))
    except UnidentifiedImageError as e:
        print(f"Could not identify image file {image_path}")
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

In [13]:
# process the images in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    executor.map(process_image, image_paths, [path_to_processed_images] * len(image_paths))

In [14]:
path_to_images = "E:\\dspro2\\Paper_data\\im2gps3ktest\\images"
path_to_processed_images = "E:\\dspro2\\Paper_data\\im2gps3ktest\\image_processed"
image_paths = [os.path.join(path_to_images, f) for f in os.listdir(path_to_images) if os.path.isfile(os.path.join(path_to_images, f))]

print(f"{len(image_paths)} files")

3000 files


In [15]:
# process the images in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    executor.map(process_image, image_paths, [path_to_processed_images] * len(image_paths))

: 